In [1]:
import sys
sys.path.insert(0, "..")
from gp3.inference import Vanilla, MultitaskLowRank
from gp3.utils.structure import kron, kron_list
from gp3.kernels import RBF
from gp3.utils.optimizers import SGD, Adam
import gp3.utils.data as sim
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from plotly import tools
from IPython.display import display
init_notebook_mode(connected=True)
from gp3.utils.transforms import softmax
from scipy.special import expit
import warnings
warnings.filterwarnings('ignore')
from tqdm import trange
import numpy as np
from gp3.utils.transforms import softplus, unit_norm
import collections

In [2]:
X = sim.sim_X_equispaced(D = 2, N_dim = 20, lower=0, upper=100)
kern_s = RBF(20., 1.)
K_s = kern_s.eval(kern_s.params, X)

n_tasks = 3
q_dim = 2
Q = unit_norm(np.random.normal(size = (n_tasks, q_dim)))
K_t = np.dot(Q, Q.T) #+ np.diag(np.ones(n_tasks))
K_full = kron(K_t, K_s)
K_full = K_full + np.diag(np.ones(K_full.shape[0]))*1e-6
K_full_chol = np.linalg.cholesky(K_full)
f = np.dot(K_full_chol, np.random.normal(size = K_full.shape[0]))
y = f + np.random.normal(size = len(f))*1.
task_idx = [int(i/400) for i in range(len(y))]

In [3]:
K_t

array([[ 1.        , -0.92548045, -0.38455958],
       [-0.92548045,  1.        ,  0.70556845],
       [-0.38455958,  0.70556845,  1.        ]])

In [ ]:
for i in range(0, len(y), int(len(y)/n_tasks)):
    fig = tools.make_subplots(rows=1, cols=2, specs=[[{'is_3d': True}, {'is_3d': True}]])
    fig.append_trace(go.Scatter3d(x=X[:,0], y=X[:,1], z=f[i:int(i+len(y)/n_tasks)],
                       mode = 'markers', marker=dict(size = 2,)), 1, 1)
    fig.append_trace(go.Scatter3d(x=X[:,0], y=X[:,1], z=y[i:int(i+len(y)/n_tasks)],
                       mode = 'markers', marker=dict(size = 2,)), 1, 2)
    iplot(fig)

This is the format of your plot grid:
[ (1,1) scene1 ]  [ (1,2) scene2 ]



This is the format of your plot grid:
[ (1,1) scene1 ]  [ (1,2) scene2 ]



This is the format of your plot grid:
[ (1,1) scene1 ]  [ (1,2) scene2 ]



In [ ]:
inf = MultitaskLowRank(X, y, kernels=[RBF(15.,1.), RBF(15.,1.)], task_idx=task_idx, noise=2.)
inf.optimizer = SGD(step_size=1e-5, momentum=0.8, decay=0.999)
inf.max_grad = 1e3
losses = inf.optimize(500)
iplot([go.Scatter(x=list(range(len(losses))), y=losses)])

Widget Javascript not detected.  It may not be installed or enabled properly.


In [ ]:
from gp3.utils.transforms import softmax, unit_norm
from scipy.special import expit

q = inf.kernels[0].unpack_params(inf.kernels[0].params)
print(q.dot(q.T))

In [ ]:
K_t

In [ ]:
pred = inf.predict_mean()
for i in range(0, len(y), int(len(y)/n_tasks)):
    fig = tools.make_subplots(rows=1, cols=3, specs=[[{'is_3d': True}, {'is_3d': True}, {'is_3d': True}]])
    fig.append_trace(go.Scatter3d(x=X[:,0], y=X[:,1], z=f[i:int(i+len(y)/n_tasks)],
                       mode = 'markers', marker=dict(size = 2,)), 1, 1)
    fig.append_trace(go.Scatter3d(x=X[:,0], y=X[:,1], z=y[i:int(i+len(y)/n_tasks)],
                       mode = 'markers', marker=dict(size = 2,)), 1, 2)
    fig.append_trace(go.Scatter3d(x=X[:,0], y=X[:,1], z=pred[i:int(i+len(y)/n_tasks)],
                       mode = 'markers', marker=dict(size = 2,)), 1, 3)
    iplot(fig)